<a href="https://colab.research.google.com/github/kikiru328/Pneumonia-Detection/blob/main/Unet%EC%9D%80%EB%82%B4%EB%88%88%EB%AC%BC%EC%9D%B4%EB%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model U-net

###### import modules

In [7]:
# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Deeplearning model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets

###### setting hyper parameters

In [11]:
cd /content/drive/MyDrive/UnetTest

/content/drive/MyDrive/UnetTest


In [12]:
lr = 1e-3
batch_size = 4
num_epoch = 100

# 학습할 데이터가 저장된 경로
data_dir = '/content/drive/MyDrive/UnetTest'

# 학습된 데이터가 저장될 경로
ckpt_dir = './checkpoint'

# 텐서보드 로그파일 저장될 경로
log_dir = './log'

# gpu / cpu 어떤걸 쓸지 device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Unet Modeling

In [14]:
# nn.Module 을 Unet 클래스에 상속시킴
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()



        # U-net 첫번째 단계 파란색 화살표 convolution 에 대한 정의
        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True): # 총 6개의 argument로 구성된 2D function  적용
            layers = []
            
            # Convolution layer 정의
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)]

            # batch normalize layer 정의
            layers += [nn.BatchNorm2d(num_features=out_channels)]

            # Relu layer 정의
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        # 해당 initial 함수에서는 해당 Unet을 정의하는데 있어 필요한 레이어들을 선언을 함.
        # forward는 init 함수에서 생성한 레이어들을 연결하는 함수.
        

        # 첫번째 파란색 화살표. 첫번째 Encoder (Enc) , stage 1 , first CBR
        # Contracting path
        # kernel_size, stride, padding, bias 는 CBR layer에서는 고정. --> Predifined이 되어있음. 따라서 in_channels ,out_channels만 사용하게됨.
        # self.enc1_1 = CBR2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        self.enc1_1 = CBR2d(in_channels=1, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)
       

        # 두번째 빨간색 화살표 , Maxpooling part (2x2)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # 두번째 파란색 화살표. layers
        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=64, out_channels=128)

        # 다음 빨간색 화살표 maxpooling part (2x2)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # 세번째 파란색 화살표 layers
        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        # 다음 빨간색 화살표  maxpooling part (2x2)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # 네번째 파란색 화살표 layers
        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)  
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        # 다음 빨간색 화살표 maxpooling part (2x2)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        # Encoder part의 마지막 CBR layer
        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # Decoder part
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        # 다음 초록색 화살표 upconvolution network (2x2) - unpooling
        # maxpool stage(4) 와 동일하게 unpool stage(4)
        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=2, stride=2, padding=0, bias=True)

        # CBR decode part
        # Encoder part 의 index 와 Decoder part의 index를 동일하게 잡아주기 위함
        # Decoder의 input_channels 와 output_channels 는 Encoder의 input_channels와 output_channels 의 정반대의 값을 보임 (거울, U)
        # Encoder part에서 하나가 추가되어 2배가 됨 ( in_channel)
        # 따라서 Decoder part의 첫번째 시작하는 입력의 채널 사이즈는 2배가 되어 진행이 되어야됨.
        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512) 
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        # 다음 초록색 화살표 upconvolution network (2x2)
        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=2, stride=2, padding=0, bias=True)

        # Decoder stage 3
        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        # 다음 초록색 화살표 upconvolution network (2x2)
        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=2, stride=2, padding=0, bias=True)

        # Decoder stage 2
        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        # 다음 초록색 화살표 upconvolution network (2x2)
        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=2, stride=2, padding=0, bias=True)

        # Decoder stage 1
        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        # 마지막으로 segmentation에 필요한 n개의 class에 대한 output을 만들기 위해 초록색 화살표 (1x1) convolution layer를 추가함.
        # fc = Final convolution
        self.fc = nn.Conv2d(in_channels=64, out_channels=2, kernel_size=1, stride=1, padding=0, bias=True)


        # 각각의 layer들을 연결하는 함수 설정
        # x = input_image
    def forward(self, x): 
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)


        #middle
        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool3(dec5_1)
        # 흰색부분 연결
        # dim = [0 : batch 방향, 1 : channel 방향, 2 : y방향(height), 3: x 방향(width)]
        cat4 = torch.cat((unpool4, enc4_2), dim=1))
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2),dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2),dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1,enc_1_2),dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x